El objetivo de este código es implementar el algoritmo MLP, separando los datos por **repetición**.
Va a buscar los datos en una carpeta que se encuentre en el mismo directorio que se llame *datos_baseline*.

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/repo_tesis/entorno_tesis_Molina"
!source bin/activate

Mounted at /content/drive
/content/drive/My Drive/repo_tesis/entorno_tesis_Molina


In [ ]:
# instalar comet ml en caso que sea necesario
!pip install comet_ml

In [3]:
# Importo las librerías
import numpy as np
from joblib import load
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import seaborn as sns
import time
from comet_ml import Experiment
import joblib
import matplotlib.pyplot as plt
import sys
sys.path.append('/content/drive/My Drive/repo_tesis/lib')
import classes as cl

In [4]:
# me situo en el directorio
%cd "/content/drive/My Drive/repo_tesis/src/baseline/datos_baseline"

# Levanto los datos
features = load("FEATURES_W200_I50.joblib")
label = load("label_W200_I50.joblib")

features_array = np.array(features)
label_array = np.array(label)

/content/drive/My Drive/repo_tesis/src/baseline/datos_baseline


In [5]:
# me situo en el directorio
%cd "/content/drive/My Drive/repo_tesis/src/baseline/datos_baseline"

# Levanto particiones
folds_ind = joblib.load('10foldsXRepeticion_W200_I50.pkl')


/content/drive/My Drive/repo_tesis/src/baseline/datos_baseline


In [6]:
for j in range(0, 10):

  # Creo una copia para no modificar el original
  folds_ind_copy = folds_ind.copy()
  print(len(folds_ind_copy))

  # Tengo que ir cambiando qué sujeto uso para test. El resto va todo para train
  folds_ind_test = folds_ind_copy[j]

  # Uno los datos de los folds de entrenamiento
  # me quedo con los folds de train quitando el de la posicion j
  folds_ind_copy.pop(j)
  folds_ind_train = folds_ind_copy.copy()

  # concateno los arrays (folds) en uno solo
  folds_ind_train_array = np.concatenate(folds_ind_train)
  # aplano el array resultante
  folds_ind_train_array = folds_ind_train_array.ravel()

  # Defino X_train, y_train, X_test, y_test
  X_train = features_array[folds_ind_train_array, :]
  y_train = label_array[folds_ind_train_array, 1]
  X_test = features_array[folds_ind_test, :]
  y_test = label_array[folds_ind_test, 1]

  start_time = time.time()    # comienzo a medir el tiempo
  # Parametros de entrada de la red neuronal que tenemos que definir
  # Definimos a mano la cantidad de features
  featureNum = 15
  chNum = 16
  # Numero de columnas que va a haber en features
  inputDim = chNum*featureNum
  # Numero de gestos contando el reposo
  clasNum = np.max(label_array[:, 1]) + 1

  # Architecture 250 - 500 - 250
  clf = cl.Clasificador(model= "mlp", arch = [250,500,250], catNum = clasNum, featureNum = inputDim)
  clf.train(X_train, y_train)

  end_time = time.time()
  training_time = end_time - start_time
  # Obtener las iteraciones del mejor modelo

  nombre = 'baseline_mlp_sep_rep' + str(j+1) + '_test'

  joblib.dump(clf, nombre + '.pkl')

  y_pred = clf.predict(X_test)

  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred, average='weighted')
  recall = recall_score(y_test, y_pred, average='weighted')
  conf_matrix = confusion_matrix(y_test, y_pred)
  plt.savefig("confusion_matrix.png")

  API_KEY = 'ehXeElNypcj7Knar5zTmyjwSO'

  exp = Experiment(api_key=API_KEY,
                  project_name='tesis-experimentos', # Nombre del proyecto donde se registran los experimentos
                  auto_param_logging=False)
  exp.set_name(nombre) # Nombre de este experimento
  exp.add_tags(['baseline', 'mlp', 'sep_rep', 'choose_test']) # Tags

  exp.log_metric("accuracy", accuracy)
  exp.log_metric("precision", precision)
  exp.log_metric("recall", recall)
  exp.log_metric("training_time", training_time)
  exp.log_confusion_matrix(y_test, y_pred)
  exp.log_parameter("sujeto_test", j)   # Guarda el arreglo en el experimento
  exp.log_text("Se usa el archivo de particion de datos 10foldsXRepeticion_W200_I50.pkl. Se guarda como parametro el numero de fold que se usa para test (se usa solo 1)")   # Comentario del experimento
  exp.log_model(name=nombre, file_or_folder=nombre + '.pkl')
  exp.end()

10
Epoch 1/50
3636/3636 [==============================] - 22s 6ms/step - loss: 1.5949 - accuracy: 0.4397
Epoch 2/50
3636/3636 [==============================] - 21s 6ms/step - loss: 1.2816 - accuracy: 0.5530
Epoch 3/50
3636/3636 [==============================] - 24s 7ms/step - loss: 1.1232 - accuracy: 0.6086
Epoch 4/50
3636/3636 [==============================] - 21s 6ms/step - loss: 1.0088 - accuracy: 0.6469
Epoch 5/50
3636/3636 [==============================] - 19s 5ms/step - loss: 0.9220 - accuracy: 0.6790
Epoch 6/50
3636/3636 [==============================] - 21s 6ms/step - loss: 0.8539 - accuracy: 0.7009
Epoch 7/50
3636/3636 [==============================] - 21s 6ms/step - loss: 0.7932 - accuracy: 0.7199
Epoch 8/50
3636/3636 [==============================] - 23s 6ms/step - loss: 0.7452 - accuracy: 0.7374
Epoch 9/50
3636/3636 [==============================] - 21s 6ms/step - loss: 0.6989 - accuracy: 0.7537
Epoch 10/50
3636/3636 [==============================] - 22s 6ms/step 

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/manuelmolinach99/tesis-experimentos/e8d1beddc84e40aea613a542dfccac4f

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : baseline_mlp_sep_rep1_test
COMET INFO:     url                   : https://www.comet.com/manuelmolinach99/tesis-experimentos/e8d1beddc84e40aea613a542dfccac4f
COMET INFO:   Metrics:
COMET INFO:     accuracy      : 0.5805384978743505
COMET INF

10
Epoch 1/50
3636/3636 [==============================] - 21s 5ms/step - loss: 1.5957 - accuracy: 0.4422
Epoch 2/50
3636/3636 [==============================] - 20s 6ms/step - loss: 1.2907 - accuracy: 0.5524
Epoch 3/50
3636/3636 [==============================] - 23s 6ms/step - loss: 1.1341 - accuracy: 0.6068
Epoch 4/50
3636/3636 [==============================] - 20s 6ms/step - loss: 1.0180 - accuracy: 0.6461
Epoch 5/50
3636/3636 [==============================] - 20s 6ms/step - loss: 0.9288 - accuracy: 0.6778
Epoch 6/50
3636/3636 [==============================] - 22s 6ms/step - loss: 0.8575 - accuracy: 0.7004
Epoch 7/50
3636/3636 [==============================] - 20s 5ms/step - loss: 0.7933 - accuracy: 0.7215
Epoch 8/50
3636/3636 [==============================] - 20s 6ms/step - loss: 0.7433 - accuracy: 0.7380
Epoch 9/50
3636/3636 [==============================] - 24s 7ms/step - loss: 0.6984 - accuracy: 0.7538
Epoch 10/50
3636/3636 [==============================] - 20s 6ms/step 

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/manuelmolinach99/tesis-experimentos/e156f05e69cc4d669c1b83b0c821fe7f

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : baseline_mlp_sep_rep2_test
COMET INFO:     url                   : https://www.comet.com/manuelmolinach99/tesis-experimentos/e156f05e69cc4d669c1b83b0c821fe7f
COMET INFO:   Metrics:
COMET INFO:     accuracy      : 0.6168003775365739
COMET INF

10
Epoch 1/50
3622/3622 [==============================] - 22s 6ms/step - loss: 1.5867 - accuracy: 0.4459
Epoch 2/50
3622/3622 [==============================] - 21s 6ms/step - loss: 1.2783 - accuracy: 0.5574
Epoch 3/50
3622/3622 [==============================] - 23s 6ms/step - loss: 1.1227 - accuracy: 0.6112
Epoch 4/50
3622/3622 [==============================] - 21s 6ms/step - loss: 1.0049 - accuracy: 0.6509
Epoch 5/50
3622/3622 [==============================] - 21s 6ms/step - loss: 0.9203 - accuracy: 0.6798
Epoch 6/50
3622/3622 [==============================] - 24s 7ms/step - loss: 0.8482 - accuracy: 0.7043
Epoch 7/50
3622/3622 [==============================] - 21s 6ms/step - loss: 0.7875 - accuracy: 0.7249
Epoch 8/50
3622/3622 [==============================] - 21s 6ms/step - loss: 0.7353 - accuracy: 0.7415
Epoch 9/50
3622/3622 [==============================] - 22s 6ms/step - loss: 0.6918 - accuracy: 0.7547
Epoch 10/50
3622/3622 [==============================] - 20s 6ms/step 

...

10
Epoch 1/50
3637/3637 [==============================] - 26s 7ms/step - loss: 1.5973 - accuracy: 0.4393
Epoch 2/50
3637/3637 [==============================] - 21s 6ms/step - loss: 1.2935 - accuracy: 0.5514
Epoch 3/50
3637/3637 [==============================] - 23s 6ms/step - loss: 1.1333 - accuracy: 0.6067
Epoch 4/50
3637/3637 [==============================] - 21s 6ms/step - loss: 1.0223 - accuracy: 0.6449
Epoch 5/50
3637/3637 [==============================] - 23s 6ms/step - loss: 0.9322 - accuracy: 0.6747
Epoch 6/50
3637/3637 [==============================] - 22s 6ms/step - loss: 0.8587 - accuracy: 0.6997
Epoch 7/50
3637/3637 [==============================] - 21s 6ms/step - loss: 0.7993 - accuracy: 0.7198
Epoch 8/50
3637/3637 [==============================] - 23s 6ms/step - loss: 0.7489 - accuracy: 0.7377
Epoch 9/50
3637/3637 [==============================] - 23s 6ms/step - loss: 0.7078 - accuracy: 0.7507
Epoch 10/50
3637/3637 [==============================] - 23s 6ms/step 

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/manuelmolinach99/tesis-experimentos/3fb7756afea84ce48d6487d5e86d20fc

COMET WARNING: Couldn't retrieve Google Colab notebook content
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : baseline_mlp_sep_rep9_test
COMET INFO:     url                   : https://www.comet.com/manuelmolinach99/tesis-experimentos/3fb7756afea84ce48d6487d5e86d20fc
COMET INFO:   Metric

10
Epoch 1/50
3624/3624 [==============================] - 23s 6ms/step - loss: 1.6000 - accuracy: 0.4402
Epoch 2/50
3624/3624 [==============================] - 22s 6ms/step - loss: 1.3010 - accuracy: 0.5474
Epoch 3/50
3624/3624 [==============================] - 21s 6ms/step - loss: 1.1441 - accuracy: 0.6037
Epoch 4/50
3624/3624 [==============================] - 23s 6ms/step - loss: 1.0250 - accuracy: 0.6441
Epoch 5/50
3624/3624 [==============================] - 21s 6ms/step - loss: 0.9333 - accuracy: 0.6747
Epoch 6/50
3624/3624 [==============================] - 21s 6ms/step - loss: 0.8641 - accuracy: 0.6988
Epoch 7/50
3624/3624 [==============================] - 23s 6ms/step - loss: 0.7979 - accuracy: 0.7193
Epoch 8/50
3624/3624 [==============================] - 21s 6ms/step - loss: 0.7504 - accuracy: 0.7364
Epoch 9/50
3624/3624 [==============================] - 23s 6ms/step - loss: 0.7044 - accuracy: 0.7505
Epoch 10/50
3624/3624 [==============================] - 22s 6ms/step 

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/manuelmolinach99/tesis-experimentos/b4362573943c40d48ace4021237b5bd5

COMET WARNING: Couldn't retrieve Google Colab notebook content
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : baseline_mlp_sep_rep10_test
COMET INFO:     url                   : https://www.comet.com/manuelmolinach99/tesis-experimentos/b4362573943c40d48ace4021237b5bd5
COMET INFO:   Metri

<Figure size 640x480 with 0 Axes>